In [2]:
from functools import cache, cached_property
import string
from typing import Literal

class Fib:
	__value: int
	def __init__(self, n: int):
		self.__value = n
	
	@property
	def value(self) -> int: return self.__value
	
	@staticmethod
	def node_id(node: int, memo: dict[int, int]) -> str:
		return memo.get(node, 1) * string.ascii_letters[node]

	@cache
	@staticmethod
	def fib(n: int) -> int:
		"""Efficient caching function to return the nth Fibonacci number."""
		if n < 2: return n
		prev, current = 0, 1
		for _ in range(n-1):
			prev, current = current, prev + current
		return current

	@cached_property
	def __memo(self) -> dict[int, int]:
		"""
		Return a dictionary of the number of times the fibonacci function is called for each n value
		when using the brute force version of the algorithm.
		"""
		result = {}
		for i in range(self.value, 0, -1):
			result[i] = Fib.fib((self.value-i) + 1)
		result[0] = Fib.fib(self.value-1)
		return result
	
	@cached_property
	def __nodes(self) -> list[str]:
		"""
		Return a list of strings representing the nodes in the mermaid graph for the tree for this
		fibonacci number.
		"""
		result = []
		for v in self.__memo:
			for j in range(1, self.__memo[v]+1):
				result.append(f"{j*string.ascii_lowercase[v]}[\"f({v})\"];")
		return result
	
	@cached_property
	def __edges(self) -> list[str]:
		"""
		Return a list of strings representing the edges in a mermaid graph for a given fibonacci memo.
		"""
		result = []
		parents = {}
		children = {}
		for v in self.__memo:
			if v < 2: continue
			for _ in range(1, self.__memo[v] + 1):
				result.append(f"{Fib.node_id(v, parents)} --> {Fib.node_id(v-1, children)} & {Fib.node_id(v-2, children)};")
				if v not in parents: parents[v] = 1
				parents[v] += 1
				if v-1 not in children: children[v-1] = 1
				if v-2 not in children: children[v-2] = 1
				children[v-1] += 1
				children[v-2] += 1
		return result
	
	def graph(self, direction: Literal["LR", "RL", "TB", "BT"] = "TB") -> str:
		"""
		Return a mermaid graph of the fibonacci function.
		"""
		return f"graph {direction}\n" + "\n".join(self.__nodes + self.__edges)
	
	def base64_string(self, direction: Literal["LR", "RL", "TB", "BT"] = "TB") -> str:
		"""
		Return a base64 encoded mermaid graph of the fibonacci function.
		"""
		import base64
		return base64.b64encode(self.graph(direction).encode("ascii")).decode("ascii")

In [11]:
from IPython.display import Image, display

f = Fib(5)
display(Image(url="https://mermaid.ink/img/" + f.base64_string()))

graph TB
f["f(5)"];
e["f(4)"];
d["f(3)"];
dd["f(3)"];
c["f(2)"];
cc["f(2)"];
ccc["f(2)"];
b["f(1)"];
bb["f(1)"];
bbb["f(1)"];
bbbb["f(1)"];
bbbbb["f(1)"];
a["f(0)"];
aa["f(0)"];
aaa["f(0)"];
f --> e & d;
e --> dd & c;
d --> cc & b;
dd --> ccc & bb;
c --> bbb & a;
cc --> bbbb & aa;
ccc --> bbbbb & aaa;
